# Change log

Could be a good idea to write what we changed and when/which version we are on now so we don't mix things up?
Or maybe not, do as you wish.

- version 01

Magnus - did a thing

Emily - Fixed the normalization part + imputer + other stuff

- version 02

Dana - introduced the one-hot encoding for preprocessing + added confusion matrix. Combining the models made by us all with the assignment detailes and providing the flow.

Magnus - started on the codes for svm and logistic regression (changed from linear regression which is only for regression, not classification) + added the lines for the model to be saved and exported into a file

- version 03 -

magnus - RandomSearchCV is implemented for all classification models, all are working

- alt version 01 -

magnus - if the data set columns has to stay the same, this is a quick fix

### Definitions 
Classification predictive modeling is the task of approximating a mapping function (f) from input variables (X) to discrete output variables (y). From the Input - till the output is one model, which may use different classification algorithms.

Classification algorithms (techniques) in supervised learning include: logistic regression, decision tree, random forest, gradient-boosted tree, multilayer perceptron, one-vs-rest, and Naive Bayes.

### Assignment: 

WHAT: The objective is to build a system for predicting heart disease and its intensity, based on observed attributes of a patient. You will have to decide which attributes to use, which classifiers to test, and eventually
which classifier to deploy.

HOW: You can use scikit-learn for pre-processing your data sources, building several data mining
models (at least three) for heart disease classification, and eventually creating a diagnosis tool for
heart disease that can be used by practitioners.

# Model 1. Using Cleveland dataset. 

Cleveland dataset is the most completed (only one missing value). Therefore it has been chosen for the building the classification model.

# Import resources

In [1]:
import pandas as pd
import numpy as np

from sklearn import metrics
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import RandomizedSearchCV

from pprint import pprint

In [2]:
# Get the data and change ? to nan-values
cle = pd.read_csv('processed.cleveland.csv', keep_default_na=False, na_values=["?"])

# Adjusting column names and NaN values

In [3]:
# Change num to goal
cle.rename(columns={'num': 'goal'}, inplace=True)

In [4]:
# Replacing NaN-values with most frequent
from sklearn.impute import SimpleImputer
replc = SimpleImputer(strategy= 'most_frequent')
replc = replc.fit_transform(cle)
cle = pd.DataFrame(replc, columns=cle.columns)

In [5]:
cle

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,goal
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0.0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2.0
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1.0
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0.0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,45.0,1.0,1.0,110.0,264.0,0.0,0.0,132.0,0.0,1.2,2.0,0.0,7.0,1.0
299,68.0,1.0,4.0,144.0,193.0,1.0,0.0,141.0,0.0,3.4,2.0,2.0,7.0,2.0
300,57.0,1.0,4.0,130.0,131.0,0.0,0.0,115.0,1.0,1.2,2.0,1.0,7.0,3.0
301,57.0,0.0,2.0,130.0,236.0,0.0,2.0,174.0,0.0,0.0,2.0,1.0,3.0,1.0


# Separating features and labels

In [6]:
# Everything except goal
cle_features = cle.drop(['goal'], axis=1)

# Only goal
cle_label = cle['goal']

# Normalization part

In [7]:
feat_NoNorm = cle_features.drop(['age', 'trestbps', 'chol', 'oldpeak', 'thalach'], axis=1)

In [8]:
numerical = 'age', 'trestbps', 'chol', 'thalach', 'oldpeak', 
X = cle_features[[c for c in cle_features if c in numerical]]

In [9]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
feat_norm = scaler.fit_transform(X)
feat_norm = pd.DataFrame(feat_norm, columns=X.columns)

In [10]:
cle_feat = pd.concat([feat_norm,feat_NoNorm], axis=1)
cle_feat

,age,trestbps,chol,thalach,oldpeak,sex,cp,fbs,restecg,exang,slope,ca,thal
0,0.948726,0.757525,-0.264900,0.017197,1.087338,1.0,1.0,1.0,2.0,0.0,3.0,0.0,6.0
1,1.392002,1.611220,0.760415,-1.821905,0.397182,1.0,4.0,0.0,2.0,1.0,2.0,3.0,3.0
2,1.392002,-0.665300,-0.342283,-0.902354,1.346147,1.0,4.0,0.0,2.0,1.0,2.0,2.0,7.0
3,-1.932564,-0.096170,0.063974,1.637359,2.122573,1.0,3.0,0.0,0.0,0.0,3.0,0.0,3.0
4,-1.489288,-0.096170,-0.825922,0.980537,0.310912,0.0,2.0,0.0,2.0,0.0,1.0,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,-1.046013,-1.234430,0.334813,-0.770990,0.138373,1.0,1.0,0.0,0.0,0.0,2.0,0.0,7.0
299,1.502821,0.700612,-1.038723,-0.376896,2.036303,1.0,4.0,1.0,0.0,0.0,2.0,2.0,7.0
300,0.283813,-0.096170,-2.238149,-1.515388,0.138373,1.0,4.0,0.0,0.0,1.0,2.0,1.0,7.0
301,0.283813,-0.096170,-0.206864,1.068113,-0.896862,0.0,2.0,0.0,2.0,0.0,2.0,1.0,3.0


# Train - test split

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(cle_feat, cle_label, test_size=0.3, random_state=321, stratify=cle_label)

# Three models to test

### Assignment

Task 3: (CLASSIFIERS) Select at least three classifiers
You may choose any set of classifiers for this assignment. 

Make sure you select at least three and
provide some reasoning of your choice. The classification models should be built using either all
attributes or a subset of the attributes. Similarly, you may also want to exclude some of the
datasets. If you decide to use a subset of attributes or exclude any dataset or part of a dataset,
please justify your choice carefully. Note that your justification should be motivated either
medically or based on an observation you have made when preprocessing the datasets.

### 1) Random forest classifier + Random Search Cross-validation

By using RandomSearchCV we can find which parameters that are most efficient to use.

In [12]:
from sklearn.ensemble import RandomForestClassifier

# Create an empty classifier
rfclf = RandomForestClassifier()

In [13]:
print('The parameters are:\n')

# Get a look att all the parameters that we can change
pprint(rfclf.get_params())

The parameters are:

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}





From research, the following parameters are usually what is included in the search:

In [14]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]

# Number of features to consider at every split
max_features = ['auto', 'sqrt', 'log2']

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 15, 20]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 5, 10, 15]

# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid_rf = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid_rf)

{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'sqrt', 'log2'],
 'min_samples_leaf': [1, 2, 5, 10, 15],
 'min_samples_split': [2, 5, 10, 15, 20],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}


In [ ]:
# Now we search for best hyperparameters, we already have the default base model "clf"

# Random search of parameters, using 5 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rfclf, param_distributions = random_grid_rf, n_iter = 100, cv = 5, verbose=2, random_state=1, n_jobs = -1)
rf_random.fit(X_train, y_train)

# This will take some minutes!

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  2.4min


In [ ]:
# Define the best parameters
best_n_estimators = rf_random.best_params_['n_estimators']
best_min_samples_split = rf_random.best_params_['min_samples_split']
best_min_samples_leaf = rf_random.best_params_['min_samples_leaf']
best_max_features = rf_random.best_params_['max_features']
best_max_depth = rf_random.best_params_['max_depth']
best_bootstrap = rf_random.best_params_['bootstrap']

# Show the best parameters
rf_random.best_params_

In [ ]:
# Now we use the best parameters to get a score.
rfclf_best = RandomForestClassifier(n_estimators=best_n_estimators, max_depth=best_max_depth, min_samples_split=best_min_samples_split, min_samples_leaf=best_min_samples_leaf, max_features=best_max_features, bootstrap=best_bootstrap, random_state=1)
rfclf_best.fit(X_train, y_train)

y_pred_rf =rfclf_best.predict(X_test)

print(metrics.classification_report(y_test, y_pred_rf))

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred_rf)

In [ ]:
data_to_classify = [45, 1, 2, 120, 155, 1, 0, 140, 0, 1.5, 3, 0, 3.0]
colnames = X_test.columns
sample = pd.DataFrame(data = [data_to_classify], columns = colnames)
sample
prediction = model.predict(sample)
print("The Patient has a predicted risk for Heart disease of ", prediction[0])
print(sample)

### 2) SVM + Random Search CV

One-vs-one is most frequently used in logistic regression for multi-class classification.

Same process as the random forest but for SVM

In [ ]:
from sklearn.svm import SVC 
# train the model on train set 
svmclf = SVC(decision_function_shape='ovo')

print('The parameters are:\n')
pprint(svmclf.get_params())

In [ ]:
# Code comes from:
# https://cambridgecoding.wordpress.com/2016/05/16/expanding-your-machine-learning-toolkit-randomized-search-computational-budgets-and-new-algorithms-2/

# Designate distributions to sample hyperparameters from 
np.random.seed(123)
g_range = np.random.uniform(0.0, 0.3, 5).astype(float)
C_range = np.random.normal(1, 0.1, 5).astype(float)

# Check that gamma>0 and C>0 
C_range[C_range < 0] = 0.0001

random_grid_svm = [{'C': list(C_range), 'gamma': list(g_range), 'kernel': ['linear'],'class_weight':['balanced', None]},
    {'C': list(C_range), 'gamma': list(g_range), 'kernel': ['rbf'],'class_weight':['balanced', None]},
    {'C': list(C_range), 'gamma': list(g_range), 'kernel': ['poly'],'class_weight':['balanced', None]},
    {'C': list(C_range), 'gamma': list(g_range), 'kernel': ['sigmoid'],'class_weight':['balanced', None]}]

print('The random grid parameters are:\n')
pprint(random_grid_svm)

In [ ]:
# Run randomized search
svm_random = RandomizedSearchCV(estimator = svmclf, param_distributions = random_grid_svm, n_iter=80, cv =5, verbose=2, n_jobs = -1)
svm_random.fit(X_train, y_train)

In [ ]:
# Identify optimal hyperparameter values
best_gamma  = svm_random.best_params_['gamma']
best_C      = svm_random.best_params_['C']
best_kernel = svm_random.best_params_['kernel']
best_class_weight = svm_random.best_params_['class_weight']

print("The best performing gamma value is: {:5.2f}".format(best_gamma))
print("The best performing C value is: {:5.2f}".format(best_C))
print("The best performing kernel is: {}".format(best_kernel))
print("The best performing class weight is: {}".format(best_class_weight))

In [ ]:
svmclf_best = SVC(kernel=best_kernel, decision_function_shape='ovo', C=best_C, gamma=best_gamma, class_weight=best_class_weight)
svmclf_best.fit(X_train, y_train)
y_pred_svm = svmclf_best.predict(X_test)

print(classification_report(y_test, y_pred_svm))

In [ ]:
confusion_matrix(y_test, y_pred_svm)

### 3) Logistic* regression + Random Search CV
#### ! Changed to logistic bc linear regression seems to only be used for regressions and not classifications, while logistic can be used for both.

One-vs-rest is most frequently used in logistic regression for multiclassification
Same process as the random forest but for linear regression

In [ ]:
from sklearn.linear_model import LogisticRegression

lrclf = LogisticRegression(multi_class='ovr', max_iter=2000)
# lrclf.fit(X_train, y_train)
print('The parameters are:\n')
pprint(lrclf.get_params())

In [ ]:
# C = np.logspace(0, 4, num=10)
# penalty = ['l1', 'l2']
# solver = ['liblinear', 'saga']
# hyperparameters = dict(C=C, penalty=penalty, solver=solver)

# randomizedsearch = RandomizedSearchCV(logistic, hyperparameters)
# best_model_random = randomizedsearch.fit(features, target)
# print(best_model_random.best_estimator_)


penalty = ['none', 'l1', 'l2', 'elasticnet']
solver = ['newton-cg', 'lbfgs', 'liblinear', 'saga']
C_lr = np.logspace(0, 4, num=10)

random_grid_lr = {'penalty': penalty,
                 'solver': solver,
                 'C': C_lr}
print('The random grid parameters are:\n')
pprint(random_grid_lr)

In [ ]:
lr_random = RandomizedSearchCV(estimator = lrclf, param_distributions = random_grid_lr, n_iter=100, cv =5, verbose=2, n_jobs = -1)
lr_random.fit(X_train, y_train)

In [ ]:
best_penalty  = lr_random.best_params_['penalty']
best_solver      = lr_random.best_params_['solver']
best_C_lr = lr_random.best_params_['C']

print("The best performing penalty is: {}".format(best_penalty))
print("The best performing solver is: {}".format(best_solver))
print("The best performing C is: {}".format(best_C_lr))

In [ ]:
lrclf_best = LogisticRegression(multi_class='ovr', penalty=12, C=0.9948482279060615, solver=lbfgs, max_iter=2000)
lrclf_best.fit(X_train, y_train)
y_pred_lr = lrclf_best.predict(X_test)

print(classification_report(y_test, y_pred_lr))

In [ ]:
confusion_matrix(y_test, y_pred_lr)

## Saving the best model

In [ ]:
import pickle
import joblib
X_train, X_test, y_train, y_test = train_test_split(cle_feat, 
                                                    cle_label, 
                                                    test_size=0.3, 
                                                    random_state=321, # Match the random state with above
                                                    stratify=cle_label)
model=lrclf_best # Change to best performing model here
model.fit(X_train, y_train)

hd_model = make_pipeline( # Remove since this is without one-hot?
    ce.OneHotEncoder(use_cat_names=True), 
    model) 
print ('the model selecetd is ', model)
hd_model.fit(X_train, y_train)
dump(hd_model,'finalized_hd_model.pkl')
y_pred = hd_model.predict(X_test)
print (y_pred)
result = hd_model.score(X_test, y_test)
print(result)

# 2. Model 2. Using all datasets combined together. 

## Data

All the datasets  were combined in one data set. 
Several datasets have been missing data in several rows. Those rows have been removed originally by code (dropna), then just manually, with same outcomes after classification.  

In [ ]:
all = pd.read_csv('all.csv', delimiter = ';')

In [ ]:
all.head()

In [ ]:
all = all.drop(['fbs','ca','slope','thal'], axis=1)

In [ ]:
all = all.replace('?', np.nan)

In [ ]:
all.dropna(thresh=2, inplace=True)

In [ ]:
all=all.reset_index()

In [ ]:
all=all.fillna(all.median(axis=0))

In [ ]:
all.info()

In [ ]:
all.exang.unique()

In [ ]:
all['Sex'].replace({0:'f',1:'m'}, inplace=True)

In [ ]:
all['Cp'].replace({1:'typ_ang',2: 'atyp_ang',3: 'non_ang',4: 'asym'}, inplace=True)

In [ ]:
all['restecg'].replace({'0.0':'norm', '1.0':'ST_T_abnorm','2.0':'l_vent_hyp', 0.0:'norm'}, inplace=True)

In [ ]:
all['exang'].replace({'1.0':'yes','0.0':'no', 0.0:'no'}, inplace=True)

In [ ]:
all.head()

In [ ]:
all = all.drop(all.columns[0], axis=1)

In [ ]:
x = all.drop(all.columns[[-1]], axis=1)

In [ ]:
y = all.iloc[:, -1]

In [ ]:
y

In [ ]:
x.head()

In [ ]:
enc_df = pd.DataFrame(enc.fit_transform(x[['Sex','Cp','restecg','exang','Country']]).toarray())

In [ ]:
enc_df.head(672)

In [ ]:
enc_df.columns = ["female", "male", "asym", "atyp_ang", 'non_ang',
                  'typ_ang','ST_T_abnorm','l_vent_hyp','restecg_norm',
                  'exang_no', 'exang_yes','cali','cle','hung','switz']

In [ ]:
x.head()

In [ ]:
x = x.drop(['Sex','Cp','restecg','exang','Country'], axis=1)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
feat_norm = scaler.fit_transform(x)
feat_norm = pd.DataFrame(feat_norm, columns=x.columns)

In [ ]:
feat_norm.head()

In [ ]:
x = pd.concat([x, enc_df], axis=1)

In [ ]:
x.head()

## Classifier Random forests

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=123, stratify=y)

In [ ]:
clf = RandomForestClassifier(n_estimators=250, random_state=123, min_samples_leaf=5)

In [ ]:
clf.fit(X_train,y_train)

In [ ]:
y_pred_all=clf.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
accuracy_score(y_test, y_pred_all)

In [ ]:
rfc_cv_score = cross_val_score(clf, x, y, cv=10)

In [ ]:
print("=== Confusion Matrix ===")
print(confusion_matrix(y_test, y_pred_all))
print('\n')
print("=== Classification Report ===")
print(classification_report(y_test, y_pred_all))
print('\n')
print("=== All AUC Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ", rfc_cv_score.mean())

The outcomes for the rf classifier are lower than with the Cleveland data sets. 
As the assignments states the model should help as a  diagnosis tool for heart disease that can be used by practitioners, building a model using all datasets has not been chosen as the final one (the decision was also supported by the voting in the group)

# Model 3. Whether the patient has heart disease. 

The classification will be higher if we would only predict whether each patient has any heart disease. 
In this case the goal will display: 0 = absence 1,2,3,4 - presence of the heart disease. 

In [ ]:
cle_hd = pd.DataFrame(replc, columns=cle.columns)

In [ ]:
cle_hd['sex'].replace({0:'f',1:'m'}, inplace=True)
cle_hd['cp'].replace({1:'typ_ang',2: 'atyp_ang',3: 'non_ang',4: 'asym'}, inplace=True)
cle_hd['fbs'].replace({1:'true', 0:'false'}, inplace=True)
cle_hd['restecg'].replace({0:'norm', 1:'ST_T_abnorm',2:'l_vent_hyp'}, inplace=True)
cle_hd['exang'].replace({1:'yes',0:'no'}, inplace=True)
cle_hd['slope'].replace({1:'upslop',2:'flat',3:'downslop'}, inplace=True)
cle_hd['thal'].replace({3.0:'normal',6.0:'fixed defect',7.0:'reversable defect'}, inplace=True)
cle_hd['goal'].replace({4.0: 1, 3.0:1, 2.0:1, 0.0: 0}, inplace=True)

In [ ]:
cle_hd.head()

In [ ]:
cle_hd.restecg.unique()

In [ ]:
cle_hd_features = cle_hd.drop(['goal'], axis=1)

In [ ]:
cle_hd_label = cle_hd['goal']

In [ ]:
#recheck
enc = OneHotEncoder(handle_unknown='ignore')
enc_df = pd.DataFrame(enc.fit_transform(cle_hd_features[['sex','cp','fbs','restecg','exang','slope']]).toarray())

In [ ]:
enc_df.columns = ["female", "male", "asym", "atyp_ang", 'non_ang',
                  'typ_ang','fbs_false','fbs_true','ST_T_abnorm','l_vent_hyp','restecg_norm',
                  'exang_no', 'exang_yes','upslop','flat','downslop','thal_fix','thal_normal','thal_reversed']

In [ ]:
cle_hd_features = pd.concat([cle_hd_features, enc_df], axis=1)
cle_hd_features = cle_hd_features.drop(['sex','cp','fbs','restecg','exang','slope', 'thal'], axis=1)
cle_hd_features['ca'] = pd.to_numeric(cle_hd_features['ca'])

In [ ]:
#recheck
feat_onehot_hd = cle_hd_features.drop(['age', 'trestbps', 'chol', 'oldpeak', 'thalach'], axis=1)

In [ ]:
numerical_hd = 'age', 'trestbps', 'chol', 'thalach', 'oldpeak', 
X_hd = cle_hd_features[[c for c in cle_features if c in numerical]]

In [ ]:
scaler_hd = StandardScaler()
feat_norm_hd = scaler_hd.fit_transform(X_hd)
feat_norm_hd = pd.DataFrame(feat_norm_hd, columns=X_hd.columns)

In [ ]:
cle_feat_hd = pd.concat([feat_norm_hd,feat_onehot_hd], axis=1)
cle_feat_hd

In [ ]:
from sklearn.model_selection import train_test_split
X_train_hd, X_test_hd, y_train_hd, y_test_hd = train_test_split(cle_feat_hd, cle_hd_label, test_size=0.3, random_state=123, stratify=cle_label)

In [ ]:
rfclf = RandomForestClassifier()

In [ ]:
pprint(rfclf.get_params())

In [ ]:
rfclf.fit(X_train_hd, y_train_hd)

In [ ]:
y_pred_hd=rfclf.predict(X_test_hd)

In [ ]:
accuracy_score(y_test_hd, y_pred_hd)

In [ ]:
rfc_hd_cv_score = cross_val_score(rfclf, cle_feat_hd, cle_hd_label, cv=10)

In [ ]:
print("=== Confusion Matrix ===")
print(confusion_matrix(y_test_hd, y_pred_hd))
print('\n')
print("=== Classification Report ===")
print(classification_report(y_test_hd, y_pred_hd))
print('\n')
print("=== All AUC Scores ===")
print(rfc_hd_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ", rfc_hd_cv_score.mean())

### Although this classification model represents the highest possible impact on the practicioner work.  
### According to the model evaluation 82.8% of patients can be diagnosed as not having any heart condition, therefore the practitioner may pay attention to the 16.2% of patients, requiring more specified diagnose. 

## The assignment description requires both:
     1. to build a system for predicting heart disease and its intensity
     2. creating a diagnosis tool for heart disease that can be used by practitioners.
    
## The group has came to a conclusion to use the best performing model 1, with the best performing classifier Logistic* regression + Random Search CV. 